In [22]:
!pip install pyautogen
!pip install python-dotenv

In [23]:
import os
import time
from google.colab import userdata
from autogen import ConversableAgent, GroupChat, GroupChatManager

**Environment variables for Gemini and DeepSeek**

In [24]:
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
os.environ['DEEPSEEK_API_KEY'] = userdata.get('DEEPSEEK_API_KEY')

**Config for Gemini**

In [25]:
config_list_gemini = [
    {
        "model": "gemini-1.5-flash",
        "api_key": os.getenv("GOOGLE_API_KEY"),
        "api_type": "google",
    }
]

**Config for DeepSeek**

In [32]:
config_list_deepseek = [
    {
        "model": "deepseek-coder:deepseek-coder-1.5-chat",
        "api_key": os.getenv("DEEPSEEK_API_KEY"),
        "api_type": "deepseek",
    }
]


**Agent -- Arguing for Electric Vehicles (EVs)**

In [26]:
ev_agent = ConversableAgent(
    name="ev_advocate",
    system_message=(
        "You are an expert in sustainability and clean energy. "
        "You passionately advocate for Electric Vehicles (EVs) as the future of transportation. "
        "Highlight their environmental benefits, technological advancements, and long-term cost savings."
    ),
    llm_config={"config_list": config_list_gemini},
    human_input_mode="NEVER",
)
ev_agent.description = "Electric Vehicle advocate using Gemini"

**Agent -- Arguing for Fuel-Powered Cars**

In [27]:
fuel_agent = ConversableAgent(
    name="fuel_defender",
    system_message=(
        "You are an automotive engineer and energy economist. "
        "You strongly support the continued use of fuel-powered cars, emphasizing current infrastructure, performance, range, and global energy needs. "
        "Argue that EVs are not yet a complete replacement for fuel-powered cars."
    ),
    llm_config={"config_list": config_list_deepseek},
    human_input_mode="NEVER",
)
fuel_agent.description = "Fuel car supporter using DeepSeek"

**Neutral judge agent**

In [28]:
judge_agent = ConversableAgent(
    name="judge_agent",
    system_message=(
        "You are a neutral and fair judge moderating a debate on whether Electric Vehicles should replace fuel-powered cars. "
        "Listen carefully to both sides and declare a winner at the end. When you have made your decision, say 'That's enough!' and explain why."
    ),
    llm_config={"config_list": config_list_gemini},  # You can switch to DeepSeek if needed
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "That's enough!" in msg["content"],
)
judge_agent.description = "Neutral judge using Gemini"

**Group Chat Configuration**

In [29]:
group_chat = GroupChat(
    agents=[ev_agent, fuel_agent, judge_agent],
    messages=[],
    send_introductions=True,
    speaker_selection_method="auto",
    max_round=20
)

**Group Chat Manager**

In [33]:
group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config={"config_list": config_list_gemini},
)

**Start Debate**

In [30]:
debate_result = safe_initiate_chat(
    judge_agent,
    group_chat_manager,
    message=(
        "Debate Topic: Should Electric Vehicles replace fuel-powered cars? "
        "Judge must listen to arguments from both sides and declare a winner by saying 'That's enough!'"
    ),
    retries=3,
    wait=15
)

judge_agent (to chat_manager):

Debate Topic: Should Electric Vehicles replace fuel-powered cars? Judge must listen to arguments from both sides and declare a winner by saying 'That's enough!'

--------------------------------------------------------------------------------

Next speaker: judge_agent

judge_agent (to chat_manager):

Understood.  I am ready to hear the arguments from both ev_advocate and fuel_defender.  Let's begin.


--------------------------------------------------------------------------------

Next speaker: judge_agent

judge_agent (to chat_manager):

Okay, ev_advocate, you may begin your opening statement.  Please present your arguments for why Electric Vehicles should replace fuel-powered cars.


--------------------------------------------------------------------------------

Next speaker: judge_agent

judge_agent (to chat_manager):

ev_advocate: Thank you.  The overwhelming evidence points to the urgent need to replace fuel-powered cars with electric vehicles. 

**Print Debate Script**

In [31]:
for idx, msg in enumerate(group_chat.messages):
    print(f"\n🔹 Round {idx + 1} — {msg['name']}:\n{msg['content']}")


🔹 Round 1 — judge_agent:
Debate Topic: Should Electric Vehicles replace fuel-powered cars? Judge must listen to arguments from both sides and declare a winner by saying 'That's enough!'

🔹 Round 2 — judge_agent:
Understood.  I am ready to hear the arguments from both ev_advocate and fuel_defender.  Let's begin.


🔹 Round 3 — judge_agent:
Okay, ev_advocate, you may begin your opening statement.  Please present your arguments for why Electric Vehicles should replace fuel-powered cars.


🔹 Round 4 — judge_agent:
ev_advocate: Thank you.  The overwhelming evidence points to the urgent need to replace fuel-powered cars with electric vehicles.  The climate crisis demands a rapid transition away from fossil fuels, and transportation is a major contributor to greenhouse gas emissions. EVs produce zero tailpipe emissions, significantly reducing our carbon footprint.  Furthermore, EVs offer superior performance in many aspects: quicker acceleration, quieter operation, and lower maintenance costs